# 페이스북 크롤링

In [165]:
# 페이지 스크롤 횟수 설정
num_scrolldown = 3

In [166]:
from bs4 import BeautifulSoup as bs
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [167]:
import pandas as pd

In [168]:
# # 로그인 페이지 접속(2단계 인증으로 로그인 불가능)

# # 로그인 정보
# username = 'global.gyeonggido@gmail.com'
# password = 's80083044!'

# element_id = driver.find_element_by_xpath('//*[@id="login_form"]/div[2]/div[1]/label/input')
# element_id.send_keys(username)
# element_password = driver.find_element_by_xpath('//*[@id="login_form"]/div[2]/div[2]/label/input')
# element_password.send_keys(password)
# time.sleep(1.5)

# ##로그인버튼 클릭
# driver.find_element_by_xpath('//*[@id="login_form"]/div[2]/div[3]/div/div/div[1]/div/span/span').click()
# time.sleep(3)

In [169]:
# 크롤링할 웹사이트 주소
url = 'https://www.facebook.com/GyeonggiKorea.en'

# webdriver 로드
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(3)
driver.implicitly_wait(5)

# 창 최대화
driver.maximize_window()
time.sleep(1)
driver.implicitly_wait(5)

In [170]:
# 페이지 스크롤 다운
for i in range(num_scrolldown):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.implicitly_wait(5)

In [171]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
print(soup.title.text)

# 팔로워 수 수집
# num_follower = soup.find('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw oo9gr5id hzawbc8m')

# 글 날짜 수집
info_dates_span = soup.find_all('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw')

# 글 내용 크롤링
content_pages = soup.find_all('div', class_='kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q')

# 좋아요, 댓글, 공유 수 정보 나온 페이지 수집
info_pages = soup.find_all('div', 'bp9cbjyn m9osqain j83agx80 jq4qci2q bkfpd7mw a3bd9o3v kvgmc6g5 wkznzc2l oygrvhab dhix69tm jktsbyx5 rz4wbd8a osnr6wyh a8nywdso s1tcr66n')

print("날짜 수집 개수 : ", len(info_dates_span))
print("글 내용 수집 개수 : ", len(content_pages))
print("각종 정보 수집 개수 : ", len(info_pages))

Gyeonggi Korea | Facebook
날짜 수집 개수 :  13
글 내용 수집 개수 :  12
각종 정보 수집 개수 :  12


In [172]:
# 팔로워수, 글 날짜, 글 내용, 좋아요 수, 댓글 수, 공유 수 리스트화
# num_follower_list = []
num_date_list = []
contents_list = []
num_like_list = []
num_comment_list = []
num_share_list = []
link_list = []

# 팔로워 수 수집(로그인되면 코드 활성화)
# num_follower = soup.find('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw oo9gr5id hzawbc8m')
# temp = float(num_follower.text[4:-2]) * 10000
# num_follower_list.append(temp)
# for i in range(len(info_pages)-1):
#     num_follower_list.append(0)

# 날짜 수집
for date in info_dates_span:
    if date.find('span') != None:
        temp = date.find('span').text[:-8]
        raw_date = temp.replace('일', '').replace(' ', '').split('월')
        month = raw_date[0]
        day = raw_date[1]
        temp_date = f'{month:0>2}-{day:0>2}'
        num_date_list.append(temp_date)

# 글 내용 수집
for content in content_pages:
    temp = content.text
    contents_list.append(temp)

# 좋아요 수 수집
for like in info_pages:
    temp = like.find('span', 'pcp91wgn')
    temp = int(temp.text)
    num_like_list.append(temp)

# 댓글 수, 공유 수 수집
for comment in info_pages:
    temp = comment.find_all('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')
    if len(temp) == 1:
        for one in temp:
            if '댓글' in one.text:
                comment_one = int(one.text[3:-1])
                num_comment_list.append(comment_one)
                num_share_list.append(0)
            elif '공유' in one.text:
                comment_share = int(one.text[3:-1])
                num_share_list.append(comment_share)
                num_comment_list.append(0)
    elif len(temp) == 2:
        for one in temp:
            if '댓글' in one.text:
                comment_one = int(one.text[3:-1])
                num_comment_list.append(comment_one)
            elif '공유' in one.text:
                comment_share = int(one.text[3:-1])
                num_share_list.append(comment_share)
    else:
        num_share_list.append(0)
        num_comment_list.append(0)

# 게시글 링크 수집
raw_link = soup.find_all('span', class_='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d9wwppkn mdeji52x e9vueds3 j5wam9gi b1v8xokw m9osqain hzawbc8m')
for link in raw_link:
    temp_link = link.find('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw').attrs['href'].split('?')[0]
    link_list.append(temp_link)

print('날짜 리스트 원소수 : ',len(num_date_list))
print('글 내용 리스트 원소수 : ',len(contents_list))
print('좋아요 수 리스트 원소수 : ',len(num_like_list))
print('공유 수 리스트 원소수 : ',len(num_share_list))
print('댓글 수 리스트 원소수 : ',len(num_comment_list))
# print('팔로워 수 리스트 원소수 : ',len(num_follower_list))
print('링크 리스트 원소수 : ',len(link_list))

날짜 리스트 원소수 :  12
글 내용 리스트 원소수 :  12
좋아요 수 리스트 원소수 :  12
공유 수 리스트 원소수 :  12
댓글 수 리스트 원소수 :  12
링크 리스트 원소수 :  12


In [173]:
# 수집 데이터를 데이터 셋으로 만들기
df = {'날짜':num_date_list, '글_내용':contents_list, '좋아요_수':num_like_list,
        '공유수':num_share_list, '댓글수':num_comment_list, '링크':link_list}
data = pd.DataFrame(df)

In [174]:
data

,날짜,글_내용,좋아요_수,공유수,댓글수,링크
0,03-12,Gyeonggi Province’s Suwon City creates “Foreig...,20,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
1,03-11,Catch this week's #GTV English #News for the l...,8,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
2,03-10,Gyeonggi Province's Hanam City home to new LG-...,21,1,0,https://www.facebook.com/GyeonggiKorea.en/post...
3,03-08,Dongducheon Rock Festival selected as an outst...,7,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
4,03-04,Gyeonggi Province’s Pocheon City provides fore...,21,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
5,03-04,Catch this week's #GTV English #News for the l...,10,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
6,03-02,Gyeonggi Province promotes local tourism with ...,22,2,0,https://www.facebook.com/GyeonggiKorea.en/post...
7,02-28,Gyeonggi Province is taking the initiative in ...,52,11,0,https://www.facebook.com/GyeonggiKorea.en/post...
8,02-25,Catch this week's #GTV English #News for the l...,11,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
9,02-24,Hwaseong Fortress of Suwon City,50,10,0,https://www.facebook.com/GyeonggiKorea.en/post...


In [175]:
# 인덱스 내림차순 정렬
final_data = data.sort_index(ascending=False)
final_data

,날짜,글_내용,좋아요_수,공유수,댓글수,링크
11,02-21,Catch this week's #GTV English #News for the l...,19,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
10,02-22,"Gwacheon City of Gyeonggi Province, known as t...",17,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
9,02-24,Hwaseong Fortress of Suwon City,50,10,0,https://www.facebook.com/GyeonggiKorea.en/post...
8,02-25,Catch this week's #GTV English #News for the l...,11,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
7,02-28,Gyeonggi Province is taking the initiative in ...,52,11,0,https://www.facebook.com/GyeonggiKorea.en/post...
6,03-02,Gyeonggi Province promotes local tourism with ...,22,2,0,https://www.facebook.com/GyeonggiKorea.en/post...
5,03-04,Catch this week's #GTV English #News for the l...,10,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
4,03-04,Gyeonggi Province’s Pocheon City provides fore...,21,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
3,03-08,Dongducheon Rock Festival selected as an outst...,7,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
2,03-10,Gyeonggi Province's Hanam City home to new LG-...,21,1,0,https://www.facebook.com/GyeonggiKorea.en/post...


In [176]:
# 수집한 데이터 파일료 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'facebook_' + file_name + '.xlsx'
path = './crawling_data/facebook/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

# 코드 마지막